In [2]:
#크롤링 데이터 분석 및 elasticsearch connect 모듈 
import requests
from bs4 import BeautifulSoup
import pandas as pd
from elasticsearch import Elasticsearch, helpers
import configparser
from datetime import datetime, timedelta
import json


In [3]:
#elastic cloud connect info load
config = configparser.ConfigParser()
config.read('example.ini')

['example.ini']

In [4]:
#elastic cloud connect
es = Elasticsearch(
    cloud_id=config['ELASTIC']['cloud_id'],
    http_auth=(config['ELASTIC']['user'], config['ELASTIC']['password'])
)

C:\Users\202210829\AppData\Local\Temp\ipykernel_6780\2986887108.py:2: DeprecationWarning: The 'http_auth' parameter is deprecated. Use 'basic_auth' or 'bearer_auth' parameters instead
  es = Elasticsearch(


In [5]:
present_date =  str(datetime.utcnow() + timedelta(hours=9))[:10]
_index = "yes24_best_100_"

In [6]:
if es.indices.exists(index = _index + present_date):
    pass
else:

    resp = es.indices.create(index=_index + present_date, body={

    "settings": {
        "analysis": {
            "analyzer": {
                "nori": {
                    "tokenizer": "nori_tokenizer"
                }
            }
        }
    },
    "mappings": {
        "properties": {
            "yes24_subject": {
                "type": "text",
                "analyzer": "nori",
                "fields":{
                    "keyword":{
                        "type":"keyword"
                    }
                }
            },
            "yes24_genre": {
                "type": "keyword"
            },
            "yes24_writer": {
                "type": "text",
                "analyzer": "nori",
                "fields":{
                    "keyword":{
                        "type":"keyword"
                    }
                }
            },
            "yes24_publisher": {
                "type": "text",
                "analyzer": "nori",
                "fields":{
                    "keyword":{
                        "type":"keyword"
                    }
                }
            },
            "yes24_date": {
                "type": "text",
                "fields":{
                    "keyword":{
                        "type":"keyword"
                    }
                }
            },
            "yes24_rank": {
                "type": "integer"
            },
            "yes24_price": {
                "type": "integer"
            },
            "yes24_review_content": {
                "type": "integer"
            },
            "yes24_review_edit": {
                "type": "integer"
            },
            "present_date": {
                "type": "date"
            },
            "sm_genre": {
                "type": "keyword"
            }
        }
    }


    })


C:\Users\202210829\AppData\Local\Temp\ipykernel_6780\1635294577.py:5: DeprecationWarning: The 'body' parameter is deprecated and will be removed in a future version. Instead use individual parameters.
  resp = es.indices.create(index=_index + present_date, body={


In [7]:
# category_num -> 장르 이름 : 장르 코드 딕셔너리
# name_mapping -> 장르 이름 : 상명 장르 딕셔너리

category_num = {
                "로맨스":"017001046",
                "BL": "017001064",
                "만화":"017001038",
                "판타지 무협": "017001049",
                "소설": "017001045",
                "경제 경영" : "017001047",                  
                "라이트노벨" : "017001063",
                "에세이 시" : "017001050",
                "인문" : "017001048",
                "사회 정치" : "017001051",
                "자기계발" : "017001054",
                "역사" : "017001052",
                "종교" : "017001053",
                "예술 대중문화" : "017001056",
                "자연과학" : "017001055",
                "가정 살림" : "017001042",
                "건강 취미 여행" : "017001043",
                "어린이 유아" : "017001058",
                "청소년" : "017001059",
                "국어 외국어" : "017001060",
                "IT 모발일" : "017001040",
                "대학교재" : "017001041",
                "수험서 자격증" : "017001061",
                "잡지" : "017001036",
                "해외원서": "017001032",
                "오디오북": "017001032",
                "크레마 (전자책 단말기)": "017001032"
                
}

name_mapping = {
                "로맨스":"문학",
                "BL": "문학",
                "만화":"문학",
                "판타지 무협": "문학",
                "소설": "문학",
                "경제 경영" : "경제/비즈니스",
                "라이트노벨" : "문학",
                "에세이 시" : "에세이/산문",
                "인문" : "인문",
                "사회 정치" : "인문",
                "자기계발" : "인문",
                "역사" : "인문",
                "종교" : "인문",
                "예술 대중문화" : "가정/생활",
                "자연과학" : "자연/과학",
                "가정 살림" : "가정/생활",
                "건강 취미 여행" : "가정/생활",
                "어린이 유아" : "가정/생활",
                "청소년" : "가정/생활",
                "국어 외국어" : "외국어",
                "IT 모발일" : "컴퓨터/인터넷",
                "대학교재" : "강의지원도서",
                "수험서 자격증" : "강의지원도서",
                "잡지" : "국외 eBook",
                "해외원서": "국외 eBook",
                "오디오북": "국외 eBook",
                "크레마 (전자책 단말기)": "국외 eBook"
                
}

In [8]:
#장르 기반 url 생성 및 크롤링 size 조절
def create_url(category_number):
    url = "http://www.yes24.com/24/category/bestseller?CategoryNumber={0}&sumgb=07&FetchSize=80".format(category_number)
    # 80개
    return url

def create_url2(category_number):
    url = "http://www.yes24.com/24/category/bestseller?CategoryNumber={0}&sumgb=07&FetchSize=20&PageNumber=2".format(category_number)
    # 20개
    return url
    

In [9]:
#크롤링 코드
bookss = [] # 책 info 저장 리스트
for key, value in category_num.items():
    rank = 0
    crolling_url = create_url(value) #크롤링 url 생성
    html = requests.get(crolling_url).text # url에 존재하는 text 가져오기
    soup = BeautifulSoup(html, 'html.parser') # html 코드 가져오기
    books = soup.select('td.goodsTxtInfo') # 책 정보를 담고있는 테그
    
    for book in books:
        title = book.select('p:nth-child(1)')[0].text # 제목
        price = book.select('p:nth-child(3) > span.priceB')[0].text # 가격
        info = book.select('div.aupu')[0].text # 책 저자 | 출판사 | 발행 날짜
        if info.count('|') ==3: # | 구분
            pass
        else:
            writer = info.split('|')[0] # 저자
        
            company = info.split('|')[1] # 출판사

            date = info.split('|')[2] # 발행 날짜
        stars = book.select('p.review > img') # 평점 이미지
        
        star_content = 0 # 내용 평점
        star_edit = 0 # 편집 평점

        if stars==None:
                    
                star_content = 0
                star_edit = 0
                
        else:

        
            for star in stars[:5]:
                
                if star['src'] == 'http://image.yes24.com/sysimage/interface/StarOn.gif':
                    
                    star_content += 1

            for star in stars[5:]:

                if star['src'] == 'http://image.yes24.com/sysimage/interface/StarOn.gif':
        
                    star_edit += 1
        

        
        rank +=1
        bookss.append([rank,title,writer, company, date,price, star_content,star_edit,key]) # 80 add


    # 위와 동일
    crolling_url2 = create_url2(value)
    html2 = requests.get(crolling_url2).text
    soup2 = BeautifulSoup(html2, 'html.parser')
    books2 = soup2.select('td.goodsTxtInfo')

        
    for book in books2:
        title = book.select('p:nth-child(1)')[0].text
        price = book.select('p:nth-child(3) > span.priceB')[0].text
        info = book.select('div.aupu')[0].text
        
        if info.count('|') ==3:
            pass
        else:
            writer = info.split('|')[0]
        
            company = info.split('|')[1]

            date = info.split('|')[2]

       
        stars = book.select('p.review > img')
        
        star_content = 0
        star_edit = 0

        if stars==None:
                    
                star_content = 0
                star_edit = 0
                
        else:

        
            for star in stars[:5]:
                
                if star['src'] == 'http://image.yes24.com/sysimage/interface/StarOn.gif':
                    
                    star_content += 1

            for star in stars[5:]:

                if star['src'] == 'http://image.yes24.com/sysimage/interface/StarOn.gif':
        
                    star_edit += 1
        

   
        rank +=1
        
        bookss.append([rank,title,writer, company, date,price, star_content,star_edit,key]) # 20 add



In [10]:
#데이터 프레임 생성
df = pd.DataFrame(bookss, columns=['yes24_rank','yes24_subject','yes24_writer', 'yes24_publisher','yes24_date','yes24_price' ,'yes24_review_content','yes24_review_edit','yes24_genre'])


#데이터 전처리-1 : 개행 문자 제거
df['yes24_date'] = df['yes24_date'].str.replace(pat=r'[^A-Za-z0-9가-힣]', repl=r' ', regex=True)
df['yes24_subject'] = df['yes24_subject'].str.replace(pat=r'[^A-Za-z0-9가-힣]', repl=r' ', regex=True)
df['yes24_writer'] = df['yes24_writer'].str.replace(pat=r'[^A-Za-z0-9가-힣]', repl=r'', regex=True)
df['yes24_publisher'] = df['yes24_publisher'].str.replace(pat=r'[^A-Za-z0-9가-힣]', repl=r'', regex=True)
df['yes24_date'] = df['yes24_date'].str.strip()

#데이터 전처리-2 : 형식 설정
df['yes24_date'] = df['yes24_date'].str.replace(pat='년 ', repl = '-',regex=True)
df['yes24_date'] = df['yes24_date'].str.replace(pat='월', repl = '',regex=True)
df['yes24_date'] = df['yes24_date'].str.strip()


df['yes24_writer'] = df['yes24_writer'].str.rstrip("저")

for j in ['원',',']:
    df['yes24_price'] = df['yes24_price'].str.replace(pat=j, repl = '',regex=True)
    df['yes24_price'] = df['yes24_price'].str.strip()
df['yes24_price'] = pd.to_numeric(df['yes24_price'])

#데이터 전처리-3 : 필요없는 요소 제거
for i in ['eBook','EPUB','완결','COMIC','PDF','BL']:
    df['yes24_subject'] = df['yes24_subject'].str.replace(pat=i, repl=r'', regex=True)


func = lambda x: name_mapping.get(x, x) # name_mapping 딕셔러니 값으로 장르 설정

df['sm_genre'] = df.yes24_genre.map(func) # sm_gener 속성 생성
df['present_date'] = present_date #time_stamp 속성 생성

In [11]:
df

,yes24_rank,yes24_subject,yes24_writer,yes24_publisher,yes24_date,yes24_price,yes24_review_content,yes24_review_edit,yes24_genre,sm_genre,present_date
0,1,새벽 침입,지렁띠,레드베릴,2021-03,3000,4,4,로맨스,문학,2023-02-25
1,2,게임 속 오메가가 되었다 ...,함초롱,동아,2022-02,4500,4,4,로맨스,문학,2023-02-25
2,3,조건 만남 삽화본,문정민,로아,2021-05,3000,4,4,로맨스,문학,2023-02-25
3,4,토굴 공주,함초롱,제로노블,2022-12,3800,4,4,로맨스,문학,2023-02-25
4,5,세트 겨울 겨울 겨울 겨울 총2권,양과람,텐북,2022-09,6300,3,4,로맨스,문학,2023-02-25
...,...,...,...,...,...,...,...,...,...,...,...
1993,12,English for the IB MYP 2 ...,DavidRitz,DaCapoPress,2009-03,35500,0,0,크레마 (전자책 단말기),국외 eBook,2023-02-25
1994,13,The Stack and Tilt Swing ...,DavidRitz,DaCapoPress,2009-03,25000,0,0,크레마 (전자책 단말기),국외 eBook,2023-02-25
1995,14,Before Sunrise Before Sunset ...,RichardLinklater,KnopfDoubledayPublishingGroup,2008-11,16500,5,5,크레마 (전자책 단말기),국외 eBook,2023-02-25
1996,15,The Guns of August ...,BarbaraWTuchman,RandomHousePublishingGroup,2009-07,11000,0,0,크레마 (전자책 단말기),국외 eBook,2023-02-25


In [12]:
# 속성 순서 재정리
df = df[['yes24_rank','yes24_subject','yes24_writer', 'yes24_publisher','yes24_date','yes24_price' ,'yes24_review_content','yes24_review_edit','yes24_genre','sm_genre','present_date']]


In [13]:
for i in range(len(df)) :
    
    es.index(index = _index + present_date,

    document = { 
        "yes24_rank" : str(df.loc[i].yes24_rank),
        "yes24_subject" : str(df.loc[i].yes24_subject),
        "yes24_writer": str(df.loc[i].yes24_writer),
        "yes24_publisher": str(df.loc[i].yes24_publisher),
        "yes24_date": str(df.loc[i].yes24_date),
        "yes24_price": str(df.loc[i].yes24_price),
        "yes24_review_content": str(df.loc[i].yes24_review_content),
        "yes24_review_edit": str(df.loc[i].yes24_review_edit),
        "yes24_genre": str(df.loc[i].yes24_genre),
        "sm_genre" : str(df.loc[i].sm_genre),
        "present_date": str(df.loc[i].present_date)
    } 
    
    )
   